#Paquetes necesarios

In [11]:
import cv2  
import math 

from ultralytics import YOLO
import easyocr
import pytesseract
import matplotlib.pyplot as plt
import imutils
import numpy as np



In [12]:
# Función para leer matrículas usando easyOCR y procesamiento de imágenes
def readLicensePlate(image):

    # Convertir a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar un filtro Gaussiano para suavizar la imagen y reducir el ruido
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Aplicar umbralización
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Crear un objeto Reader de EasyOCR
    reader = easyocr.Reader(['en'])

    # Obtener las regiones de texto utilizando EasyOCR
    results = reader.readtext(thresh)

    print(results)

    # Imprimir los resultados
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        cv2.rectangle(image, tuple(map(int, top_left)), tuple(map(int, bottom_right)), (0, 255, 0), 2)
        cv2.putText(image, text, tuple(map(int, top_left)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Mostrar la imagen con los resultados
    # cv2.imshow('OCR Result', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # if len(result) > 0: 
    #     print("Resultado: " + result[0][1])
    #     return result[0][2]
    # else: 
    #     print("No se ha encontrado texto.")
    #     return None
# img = cv2.imread('images/images/Cars3.png')
# readLicensePlate(img)


In [13]:
# Función para determinar la ubicación de la matrícula
def searchLicensePlate(image, coords):
    subImg = image[coords[2]:coords[3], coords[0]:coords[1]]
    subImg = resizeImage(subImg, 0.2, 0.1, 0.4, 0.2)
    return subImg

# Función para redimensionar una imagen porcentualmente por cada uno de sus 4 lados
def resizeImage(image, leftToRight, rightToLeft, topToBottom, bottomToTop):
    sizes = image.shape
    subImg = image[round(sizes[0]*topToBottom):sizes[0]-round(sizes[0]*bottomToTop),round(sizes[1]*leftToRight):sizes[1]-round(sizes[1]*rightToLeft)]

    return subImg

# Función para usar un modelo de YOLO
def useModel(pathModel, img, license):
    model = YOLO(pathModel)
    resultsModel = model(img, stream=True)
    boxesFiltered = []
    for r in resultsModel:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            if not license:
                if cls != 2:
                    continue
            boxesFiltered.append(box)
    return boxesFiltered

# Función para mostrar los resultados obtenidos de un modelo YOLO
def printResultsInImage(boxes, img):
    i = 0
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
        # Confianza
        confidence = math.ceil((box.conf[0]*100))/100
        # print("Confidence --->",confidence)

        # Clase
        cls = int(box.cls[0])

        if cls == 2:
            # print("Class name --> car")
            cv2.putText(img, 'car', [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        else:
            # print("Class name --> license")
            cv2.putText(img, 'license', [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)
            
        subImg = img[y1:y2,x1:x2]

        i += 1
        cv2.imwrite('images/result' + str(i) + '.jpg', subImg)

        # readLicensePlate(subImg)

    cv2.imwrite('images/result.jpg', img)
    cv2.imshow('images/result.jpg', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# img = cv2.imread('images/images/Cars104.png')
# boxes = useModel('yolov8n.pt', img, False)
# printResultsInImage(boxes, img)
# mixModels(img, classNamesOriginalModel)
# showResultsModel(results, classNamesOriginalModel, img)

In [21]:
# Función para mezclar modelos de YOLO, de detección de vehículos y de matrículas y así obtener resultados más precisos
def mixModels(img):
    boxesCars = useModel('yolov8n.pt', img, False)
    results = []
    imgcpy = img

    for box in boxesCars:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        x1_1, y1_1, x2_1, y2_1 = int(x1), int(y1), int(x2), int(y2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)

        subImg = img[y1:y2,x1:x2]
        # cv2.imshow('car', subImg)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        boxesLicenses = useModel('yolov8n-licenseTrained.pt', subImg, True)
        for boxL in boxesLicenses:
            x1, y1, x2, y2 = boxL.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # cv2.rectangle(subImg, (x1, y1), (x2, y2), (0, 0, 0), 3)

            licenseImg = subImg[y1:y2,x1:x2]

            imgcpy[y1_1:y2_1, x1_1:x2_1] = subImg

            readLicensePlate(licenseImg)

            # cv2.imshow('license', licenseImg)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
        results.extend(boxesLicenses)
    
    return results
        
        

# img = cv2.imread('images/images/Cars2.png')
# boxes = mixModels(img)
# printResultsInImage(boxes, img)

In [25]:
# Función para pobar el modelo mixto en un vídeo
def videoTest(pathVideo):
    cap = cv2.VideoCapture(pathVideo)

    if not cap.isOpened():
        print("Error al abrir el vídeo")    
        exit()
    
    nFrames = 0
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # mixModels(frame)
        nFrames += 1
        if nFrames % 100 == 0:
            mixModels(frame)
            print(nFrames)
            cv2.imshow('Video', frame)
            key = cv2.waitKey(0) & 0xFF
            
        
        cv2.imshow('Video', frame)

        if cv2.waitKey(3) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print(nFrames)


videoTest('videos/Traffic IP Camera.mp4')


0: 384x640 2 persons, 5 cars, 95.8ms
Speed: 0.0ms preprocess, 95.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 576x640 1 licence, 84.7ms
Speed: 12.3ms preprocess, 84.7ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[0, 0], [87, 0], [87, 32], [0, 32]], 'JA 0254S', 0.4584285027473914)]



0: 608x640 1 licence, 254.1ms
Speed: 16.9ms preprocess, 254.1ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[18, 0], [63, 0], [63, 22], [18, 22]], '164', 0.6142041683197021)]



0: 608x640 (no detections), 233.6ms
Speed: 0.0ms preprocess, 233.6ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 640x640 (no detections), 238.6ms
Speed: 3.2ms preprocess, 238.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x608 (no detections), 265.5ms
Speed: 17.7ms preprocess, 265.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 608)


...

[([[28, 4], [88, 4], [88, 30], [28, 30]], 'Y6190', 0.4674469868616977)]



0: 576x640 1 licence, 233.3ms
Speed: 17.3ms preprocess, 233.3ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[24, 0], [88, 0], [88, 33], [24, 33]], '82545', 0.3657444511665837)]



0: 608x640 (no detections), 282.7ms
Speed: 0.0ms preprocess, 282.7ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)


700
749
